<a href="https://colab.research.google.com/github/SpyrosKostis/Analyze_ab_test_results_notebook/blob/master/Get_Historical_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (15,7)

import seaborn as sns
from datetime import datetime, timedelta,date

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import requests 
from scipy import stats
import statsmodels.api as sm
from itertools import product

import warnings
warnings.filterwarnings('ignore')

In [0]:
coins=['BTC','ETH','XRP','BCH','LTC','EOS','XTZ','XLM','ETC','LINK','DASH']

In [0]:
# remove 'USD' level
#for k in coins:
  url='https://min-api.cryptocompare.com/data/v2/histohour?fsym=XMR&tsym=USD&limit=24&aggregate=3&extraParamsapi_key={5c2552fbb4c69884a6e1afab5f2bd79db15cbf9bc8874ef8cc43583c44c428e4}'
  f = requests.get(url)
  ipdata = f.json()


In [0]:
ipdata

{'Data': {'Aggregated': True,
  'Data': [{'close': 73.99,
    'conversionSymbol': 'BTC',
    'conversionType': 'multiply',
    'high': 74.61,
    'low': 73.6,
    'open': 74.48,
    'time': 1580817600,
    'volumefrom': 70991.06,
    'volumeto': 5252800.46},
   {'close': 75.4,
    'conversionSymbol': 'BTC',
    'conversionType': 'multiply',
    'high': 75.54,
    'low': 74.42,
    'open': 73.99,
    'time': 1580828400,
    'volumefrom': 17872.62,
    'volumeto': 1340962.65},
   {'close': 74.64,
    'conversionSymbol': 'BTC',
    'conversionType': 'multiply',
    'high': 75.12,
    'low': 74.16,
    'open': 75.4,
    'time': 1580839200,
    'volumefrom': 15555.06,
    'volumeto': 1160545.45},
   {'close': 75.05,
    'conversionSymbol': 'BTC',
    'conversionType': 'multiply',
    'high': 75.48,
    'low': 74.34,
    'open': 74.64,
    'time': 1580850000,
    'volumefrom': 13600.36,
    'volumeto': 1021080.5},
   {'close': 74.72,
    'conversionSymbol': 'BTC',
    'conversionType': 'mult

In [0]:
coin_data = pd.DataFrame(ipdata['Data']['Data'])
coin_data

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,1580817600,74.61,73.60,74.48,70991.06,5252800.46,73.99,multiply,BTC
1,1580828400,75.54,74.42,73.99,17872.62,1340962.65,75.40,multiply,BTC
2,1580839200,75.12,74.16,75.40,15555.06,1160545.45,74.64,multiply,BTC
3,1580850000,75.48,74.34,74.64,13600.36,1021080.50,75.05,multiply,BTC
4,1580860800,75.45,74.40,75.05,13545.31,1012212.11,74.72,multiply,BTC
5,1580871600,75.75,74.79,74.72,11558.64,868378.23,75.13,multiply,BTC
6,1580882400,75.58,75.11,75.13,12730.09,957437.28,75.19,multiply,BTC
7,1580893200,76.91,75.12,75.19,24452.97,1862753.64,76.67,multiply,BTC
8,1580904000,78.34,76.73,76.65,28731.01,2232278.83,77.57,multiply,BTC
9,1580914800,78.83,77.56,77.57,19482.09,1523910.78,78.30,multiply,BTC


In [0]:
# exclude coins that haven't traded in last 24 hours
# TOTALVOLUME24H is the amount the coin has been traded
# in 24 hours against ALL its trading pairs
coin_data = coin_data[coin_data['TOTALVOLUME24H'] != 0]

In [0]:
coin_data.head(10)

In [0]:
top_coins = coin_data[:9].index
top_coins

In [0]:
hist = cc.get_historical_price_minute('EOS', curr='USD', limit=2000)
hist_df = pd.DataFrame(hist)

In [0]:
hist_df.columns

In [0]:
def get_data(coin):
    hist = cc.get_historical_price_minute(coin, curr='USD', limit=2000)
    if hist:
        hist_df = pd.DataFrame(hist)
        hist_df['time'] = pd.to_datetime(hist_df['time'], unit='s')
        hist_df.index = hist_df['time']
        hist_df['%diff']=((hist_df['close']-hist_df['open'])/hist_df['open'])*100
        hist_df['diffhl']=((hist_df['high']-hist_df['low'])/hist_df['low'])*100
        hist_df['diffvol']=((hist_df['volumeto']-hist_df['volumefrom'])/hist_df['volumefrom'])*100
        hist_df['rol_mean_24H']=hist_df['close'].rolling(24).mean()
        hist_df['rol_mean_10H']=hist_df['close'].rolling(10).mean()
        hist_df['from_mean_24H']=hist_df['close']-hist_df['rol_mean_24H']
        hist_df['from_mean_10H']=hist_df['close']-hist_df['rol_mean_10H']
        colum=['rol_mean_10H','rol_mean_24H','close']
        for c in colum:
            hist_df['diff'+c]=hist_df[c]-hist_df[c].shift(1)
            hist_df.loc[hist_df['diff'+c]<0,'diff'+c]=-1
            hist_df.loc[hist_df['diff'+c]>0,'diff'+c]=1
            hist_df.loc[hist_df['diff'+c]==0,'diff'+c]=0
        del hist_df['time']
        for c in hist_df.columns:
            hist_df.rename(columns={c: c+coin},inplace=True)
        return(hist_df)


In [0]:
data=get_data('EOS')

In [0]:
for coin in top_coins:
    data2=get_data(coin)
    data=data.merge(data2, left_index=True,right_index=True)

In [0]:
data.head(50)

In [0]:
full_csv_m= data.to_csv (r'C:\Users\Spyros\Desktop\data analysis\crypto\mira\fullcsv_m.csv', index = True, header=True)
full_xls_m= data.to_excel (r'C:\Users\Spyros\Desktop\data analysis\crypto\mira\fullxls_m.xlsx', index = True, header=True) 